# Simulación Estocástica
## Tarea Nº 01

Alonso Ogueda Oliva

### Item 1
Escriba una rutina para resolver el sistema de ecuaciones lineales $\mathbf{Ax} = \mathbf{b}$ con:

$$\mathbf{A} = \begin{pmatrix}
1 & 1 & 1 \\
2 & 3 & 1 \\
1 & -1 & -1 \\
\end{pmatrix}
,  \qquad
\mathbf{b} = \begin{pmatrix}
4 \\
9 \\
-1
\end{pmatrix}
$$
usando la **descomposición LU** y **refinamiento iterativo.**

In [1]:
import numpy as np
from scipy import linalg as la

In [2]:
A = np.array([
    [1., 1., 1.],
    [2., 3., 1.],
    [1., -1., -1.]
])

b = np.array([4, 9, -1])

In [3]:
np.linalg.det(A)

-4.000000000000001

In [4]:
def lu_decomposition(A):
    a = A.copy().astype(float)
    n = a.shape[0]
    for k in range(n):
        if a[k, k] == 0:
            raise ValueError
        a[k + 1: n, k] = a[k + 1: n, k] / a[k, k]
        a[k + 1: n, k + 1: n] = a[k + 1: n, k + 1: n] - np.outer(a[k + 1: n, k], a[k, k + 1: n])
    l = np.tril(a, -1)
    u = np.triu(a)
    np.fill_diagonal(l, 1)
    return l, u

In [5]:
L, U = lu_decomposition(A)

Se verifica que la descomposición es correcta

In [6]:
np.allclose(A, L @ U)

True

In [7]:
def back_substitution(T, b):
    T = T.astype(float)
    n = T.shape[0]
    x = np.zeros(n)
    x[n - 1] = b[n - 1] / T[n - 1, n - 1]
    for i in range(n - 2, -1, -1):
        x[i] = (b[i] - np.inner(T[i, i:], x[i:])) / T[i, i]
    return x

In [8]:
def forward_substitution(T, b):
    T = T.astype(float)
    n = T.shape[0]
    x = np.zeros(n)
    x[0] = b[0] / T[0, 0]
    for i in range(1, n):
        x[i] = (b[i] - np.inner(T[i, :i], x[:i])) / T[i, i]
    return x

In [9]:
def lu_solve(A, b):
    L, U = lu_decomposition(A)
    x = back_substitution(U, forward_substitution(L, b))
    return x

In [10]:
x_0 = lu_solve(A, b)
print(x_0)

[1.5  1.75 0.75]


In [11]:
def iterative_refinement(A, b, x_0, k_max=100, tol=10e-6):
    A = A.astype(np.float_)
    b = b.astype(np.float_)
    x_k = x_0.astype(np.float_)
    k = 0
    converge = False
    while not converge:
        r_k = b - A @ x_k
        delta_k = lu_solve(A, r_k)
        x_k += delta_k
        if la.norm(delta_k, np.inf) <= tol * la.norm(x_k, np.inf):
            return x_k
        elif k < k_max:
            k += 1
        else:
            print("Se superó la cantidad de iteraciones.")

In [12]:
iterative_refinement(A, b, x_0, k_max=100, tol=10e-20)

array([1.5 , 1.75, 0.75])

Es posible notar que antes y después del refinamiento la solución no cambia

In [13]:
def lu_and_iterative_refinement(A, b, k_max=100, tol=10e-6):
    x_0 = lu_solve(A, b)
    x = iterative_refinement(A, b, x_0, k_max, tol)
    return x

In [14]:
lu_and_iterative_refinement(A, b, k_max=100, tol=10e-6)

array([1.5 , 1.75, 0.75])

### Item 2
Implementar el operador Sweep usando su lenguaje de programación favorito.

In [15]:
def sweep(A, k):
    if (len(set(A.shape)) != 1) or (A[k, k] == 0):
        raise ValueError("Not a square array")
    B = A - np.outer(A[:, k], A[k, :]) / A[k, k]
    B[:, k] = -1 * A[:, k] /  A[k, k]
    B[k, :] = A[k, :] /  A[k, k]
    B[k, k] = 1. / A[k, k]
    return B

### Item 3
Pruebe su rutina para obtener la inversa de una matriz usando el operador
Sweep con la siguiente matriz:

$$\mathbf{B} = \begin{pmatrix}
30 & 16 & 46 \\
16 & 10 & 26 \\
46 & 26 & 72 \\
\end{pmatrix}
$$

In [16]:
def sweep_inv(A):
    A_inv = A.copy()
    for i in range(A.shape[0]):
        A_inv = sweep(A_inv, i)
        return A_inv

In [17]:
B = np.array([
    [30., 16., 46.],
    [16., 10., 26.],
    [46., 26., 72.]
])
print(sweep_inv(B))

[[ 0.03333333  0.53333333  1.53333333]
 [-0.53333333  1.46666667  1.46666667]
 [-1.53333333  1.46666667  1.46666667]]


Note que la matriz anterior no es la inversa de $B$, en efecto:

In [18]:
sweep_inv(B) @ B

array([[ 80.06666667,  45.73333333, 125.8       ],
       [ 74.93333333,  44.26666667, 119.2       ],
       [ 44.93333333,  28.26666667,  73.2       ]])

In [19]:
B @ sweep_inv(B)

array([[ -78.06666667,  106.93333333,  136.93333333],
       [ -44.66666667,   61.33333333,   77.33333333],
       [-122.73333333,  168.26666667,  214.26666667]])

Lo cual se debe a que $\mathbf{B}$ es singular!

In [20]:
la.det(B)

0.0

### Item 4
Verifique que $\mathbf{B}$ es matriz semidefinida positiva usando la factorización Cholesky. ¿En cuál etapa del algoritmo el procedimiento falla?

In [21]:
def cholesky_decomposition(A):
    n = A.shape[0]
    T = np.zeros(A.shape)
    T[0, :] = np.concatenate([np.sqrt(A[0, [0]]), A[0, 1:] / np.sqrt(A[0, 0])])
    for i in range(1, n):
        T[i, i] = np.sqrt(A[i, i] - np.inner(T[:i, i], T[:i, i]))
        for j in range(i + 1, n):
            T[i, j] = (A[i, j] - np.inner(T[:i, i], T[:i, j])) / T[i, i]
    return T

In [22]:
G = cholesky_decomposition(B)

In [23]:
G

array([[5.47722558, 2.92118697, 8.39841255],
       [0.        , 1.21106014, 1.21106014],
       [0.        , 0.        , 0.        ]])

Note que aún así se cumple que $\mathbf{G}^\top \mathbf{G} = \mathbf{B}$.

In [24]:
np.allclose(B, (G.T @ G))

True

Pero un elemento de la diagonal es no positivo!

In [25]:
all(np.diag(G) > 0)

False

Lo anterior puede deberse a un problema de precisión o estabilidad, puesto que la función ya implementada en `scipy` entrega una descomposición con todos los valores no nulos (aunque el último muy cercano a cero).

In [26]:
la.cholesky(B)

array([[5.47722558e+00, 2.92118697e+00, 8.39841255e+00],
       [0.00000000e+00, 1.21106014e+00, 1.21106014e+00],
       [0.00000000e+00, 0.00000000e+00, 1.19209290e-07]])